In [1]:
import os
from glob import glob

import numpy as np
import xarray as xr
import dask
from regridding import regrid

import gev_utils as gevu
import gev_nonstat_utils as gevns

from utils import gev_metric_ids
from utils import roar_data_path as project_data_path
from utils import roar_code_path as project_code_path

In [2]:
# ############
# ### Dask ###
# ############
# from dask_jobqueue import SLURMCluster

# cluster = SLURMCluster(
#     account="pches",
#     # account="open",
#     cores=1,
#     processes=1,
#     memory="4GiB",
#     walltime="09:00:00",
# )

# cluster.scale(jobs=40)  # ask for jobs

# from dask.distributed import Client
# client = Client(cluster)
# client

## Fit GEV

In [ ]:
# Non-stationary, MLE, main
for metric_id in gev_metric_ids:
    gevu.gev_fit_all(
        metric_id = metric_id,
        stationary = False,
        fit_method = "lmom",
        periods_for_level = [10, 25, 50, 100],
        levels_for_period = None,
        proj_years = [2050,2100],
        hist_years = [1950,2014],
        bootstrap = False
    )

In [4]:
# Stationary, L-moments, main
for metric_id in gev_metric_ids:
    gevu.gev_fit_all(
        metric_id = metric_id,
        stationary = True,
        fit_method = "lmom",
        periods_for_level = [10, 25, 50, 100],
        levels_for_period = None,
        proj_years = [2050,2100],
        hist_years = [1950,2014],
        bootstrap = False
    )

CPU times: user 13.8 s, sys: 829 ms, total: 14.7 s
Wall time: 31.7 s


In [3]:
## Stationary, L-moments, bootstrap

# Can separate STAR-ESDM since it's higher resolution 
# and requires significantly more memory
for metric_id in gev_metric_ids:
    gevu.gev_fit_all(
        metric_id = metric_id,
        stationary = True,
        fit_method = "lmom",
        periods_for_level = [10, 25, 50, 100],
        levels_for_period = None,
        proj_years = [2050,2100],
        hist_years = [1950,2014],
        bootstrap = True,
        include_STAR_ESDM = True
    )

## Regridding

In [4]:
gev_metric_ids

['max_tasmax', 'min_tasmin', 'max_pr', 'max_cdd', 'max_hdd']

In [3]:
%%time
# Regrid to LOCA using NN
method = "nearest"
target = "LOCA2"

for metric_id in gev_metric_ids:
    # Save path
    store_path = f"{project_data_path}/extreme_value/loca_grid/{metric_id}"
    # Get all files
    files = glob(f"{project_data_path}/extreme_value/original_grid/{metric_id}/STAR-ESDM*") + \
            glob(f"{project_data_path}/extreme_value/original_grid/{metric_id}/GARD-LENS*")

    # Loop through
    for file in files:
        # Check if done
        file_name = file.split('/')[-1]
        store_name = file_name.replace(".nc", f"_{method}.nc")
        if not os.path.exists(f"{store_path}/{store_name}"):
            # Regrid and store
            ds_tmp = xr.open_dataset(file)
            ds_out = regrid(ds_tmp, method=method, target=target)
            ds_out.to_netcdf(f"{store_path}/{store_name}")

CPU times: user 55.6 s, sys: 40.2 s, total: 1min 35s
Wall time: 2min 37s


In [5]:
%%time
# Regrid to GARD-LENS using NN
method = "nearest"
target = "GARD-LENS"

for metric_id in gev_metric_ids:
    # Save path
    store_path = f"{project_data_path}/extreme_value/gard_grid/{metric_id}"
    # Get all files
    files = glob(f"{project_data_path}/extreme_value/original_grid/{metric_id}/STAR-ESDM*") + \
            glob(f"{project_data_path}/extreme_value/original_grid/{metric_id}/LOCA2*")

    # Loop through
    for file in files:
        # Check if done
        file_name = file.split('/')[-1]
        store_name = file_name.replace(".nc", f"_{method}.nc")
        if not os.path.exists(f"{store_path}/{store_name}"):
            # Regrid and store
            ds_tmp = xr.open_dataset(file)
            ds_out = regrid(ds_tmp, method=method, target=target)
            ds_out.to_netcdf(f"{store_path}/{store_name}")

CPU times: user 42.7 s, sys: 39.3 s, total: 1min 22s
Wall time: 2min 21s
